Deep Learning
=============

Assignment 1
------------

The objective of this assignment is to learn about simple data curation practices, and familiarize you with some of the data we'll be reusing later.

This notebook uses the [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) dataset to be used with python experiments. This dataset is designed to look like the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, while looking a little more like real data: **it's a harder task, and the data is a lot less 'clean' than MNIST.**

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.

# General variables
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import pickle

# iPython Packages
from IPython.display import display, Image

# ML Packages
from scipy import ndimage
from sklearn.linear_model import LogisticRegression

# Download Packages
from urllib.request import urlretrieve

# Configuration
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labelled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [ ]:
# Seed the numpy randomizer early on so we always get 
# our 'random' values back
np.random.seed(42)

In [2]:
# What directory are you working from? Let's remember that
maindir = os.getcwd()
print(maindir)

/Users/vivek.menon/Dropbox/learn/books/Udacity/UD730 - Deep Learning


In [23]:
# Some folder walking tools

def root_folder():
    os.chdir(maindir)
    
def current_folder():
    print('Current folder: {}'.format(os.getcwd().split('/')[-1]))
    return os.getcwd()
    
def data_folder():
    root_folder()
    
    if not os.path.exists('data'):
    # Make a 'data' folder to hold these datasets if doesn't exist yet
        print('Creating data folder')
        os.makedirs('data')
    else:
        os.chdir('data')
    
    

current_folder()
data_folder()
current_folder()
root_folder()

Current folder: UD730 - Deep Learning
Current folder: data


In [24]:
url = 'http://commondatastorage.googleapis.com/books1000/'
# Where are we going? 

last_percent_reported = None
# Assume we've not downloaded anything

def download_progress_hook(count, blockSize, totalSize):
    """A hook to report the progress of a download. This is mostly intended for users with
    slow internet connections. Reports every 1% change in download progress.
    """
    global last_percent_reported
    # Reference the global percent variable
    
    percent = int(count * blockSize * 100 / totalSize)
    # Take the count * blockSize / totalSize , multiply by 100 for a percent

    if last_percent_reported != percent:
    # If the precent has since changed
        if percent % 5 == 0:
            # And if the current precent is past an increment of 5
            sys.stdout.write("%s%%" % percent)
            # Print the current percent at 5% increments
            sys.stdout.flush()
            # Flsuh the terminal; doesn't seem to do anything in Jupyter
        else:
            sys.stdout.write(".")
            # Else just print a '.'
            sys.stdout.flush()
            
    last_percent_reported = percent
    # Set the last_percent_reported global to the current percent given
        
def download(filename, expected_bytes, force=False):
    """Download a file if not present, and make sure it's the right size."""
    
        
    if force or not os.path.exists(filename):      
    # If force == True or the data does not exist  
        print('Attempting to download:', filename) 
        download = urlretrieve(url + filename, filename, reporthook=download_progress_hook)
        # Download the filename, and use the download_progress_hook inteface to report back results 
        # reporthook probably hands off count, blockSize, totalSize, and other info, because its not in here
        # urlretreive returns a tuple: ('filename', <httplib.HTTPMessage instance at someaddress>) 
        # Use filename, _ to split the name from the other stuff (we use _ to store the misc stuff) 
        
        filename, _ = download
        #print('Retrieved tuple named {}, so split it into {} and {}'.format(download, filename, _))

        # Confirm completion
        print('\nDownload Complete!')
        
    statinfo = os.stat(filename)
    # Check the bytesize of our new file
    
    if statinfo.st_size == expected_bytes:
    # If the filesize matches expected bytes
        print('Found and verified', filename)
    else:
    # If it's not 
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
        # Raise an exception to tell me it didn't work
    
    return filename
    # Returns the actual name when called, but will do this whole function in an attempt to return this

    
data_folder()
# Jump into the data folder 

train_filename = download('notMNIST_large.tar.gz', 247336696)
# What is the training file

test_filename = download('notMNIST_small.tar.gz', 8458043)
# What is the testing file

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz



Extract the dataset from the compressed .tar.gz file.
This should give you a set of directories, labelled A through J.

In [27]:
def extract(filename, num_classes, force=False):
    """Extract the .tar into subfolders, and check if right number of classes."""
    
    root = os.path.splitext(os.path.splitext(filename)[0])[0]  
    # remove the file type at the '.'; first for .gz, then for .tar

    if os.path.isdir(root) and not force:
        # You may override by setting force=True.
        print('%s already present - Skipping extraction of %s.' % (root, filename))
        # The directory already exists
    else:
        print('Extracting data for %s. This may take a while. Please wait.' % root)
        tar = tarfile.open(filename)
        # open the file as a 
        sys.stdout.flush()
        # Clean the terminal output
        tar.extractall()
        # Extract the tar file into the directory
        tar.close()
        # Close the tar
    
    data_folders = [os.path.join(root, d) for d in sorted(os.listdir(root)) if os.path.isdir(os.path.join(root, d))]
    # List comprehension to get all subfolder strings from a list of all folders if they are each directories
    
    if len(data_folders) != num_classes:
    # if there are few folders than expected, raise exception
        raise Exception('Expected %d folders, one per class. Found %d instead.' % (num_classes, len(data_folders)))
    print(data_folders)
    # Say what they are
    
    return data_folders
    # return it for convenience 

data_folder()
# Jump into the data folder 

train_folders = extract(train_filename, 10)
# What are the training folders (I expect 10)

test_folders = extract(test_filename, 10)
# What are the test folders (I expect 10)

notMNIST_large already present - Skipping extraction of notMNIST_large.tar.gz.
['notMNIST_large/A', 'notMNIST_large/B', 'notMNIST_large/C', 'notMNIST_large/D', 'notMNIST_large/E', 'notMNIST_large/F', 'notMNIST_large/G', 'notMNIST_large/H', 'notMNIST_large/I', 'notMNIST_large/J']
Extracting data for notMNIST_small. This may take a while. Please wait.
['notMNIST_small/A', 'notMNIST_small/B', 'notMNIST_small/C', 'notMNIST_small/D', 'notMNIST_small/E', 'notMNIST_small/F', 'notMNIST_small/G', 'notMNIST_small/H', 'notMNIST_small/I', 'notMNIST_small/J']


---
Problem 1
---------

Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded.

**Hint**: you can use the package `IPython.display`.

---

In [88]:
def grab_sample(folder, num_images):
    """Grab a random sample of images and show them"""
    
    os.chdir(folder)
    # Get into that folder
    
    subfolder = os.listdir()[np.random.randint(0,len(os.listdir()))]
    # Pick a random subfolder

    os.chdir(subfolder)
    # Get in there
    
    image_files = [os.listdir()[i] for i in range(0,num_images)]
    # Pick a random set of image files
    
    current_folder()
    [display(Image(filename=i)) for i in image_files]
    # Show us what you picked
    
data_folder()
grab_sample('notMNIST_large',10)
data_folder()

Current folder: D


Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. Later we'll merge them into a single dataset of manageable size.

We'll convert the entire dataset into a **3D array (image index, x, y)** of floating point values, **normalized to have approximately zero mean and standard deviation ~0.5** to make training easier down the road. 

A few images might not be readable, we'll just skip them.

In [162]:
image_height = 28  # Pixel height (pre-defined)
image_width = 28 # Pixel width (pre-defined)
pixel_depth = 255.0  # Number of levels per pixel (pre-defined)

def load_letter(folder, min_num_images):   
    """Load the data for a single letter label."""
    os.chdir(folder)
    image_files = os.listdir()
    # Go into that subfolder and grab all the images 
    
    print('Creating a {} x {} x {} tensor to fill with normalized values between 0 and {}'.format(
        len(image_files), image_height, image_width, pixel_depth))
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
    # Make a dataset to fill as long as the images, with the 
    # image_height, image_weidth, and holding float32 data
    
    num_images = 0    
    # Quick counter
    
    for image in image_files:
        print('On image {}.'.format(num_images))
        try: 
            image_data = (ndimage.imread(image).astype(float) - 
                          pixel_depth / 2) / pixel_depth
            
            # Read the image_data as a float, normalizing by subtracting (max_value/2) 
            # and dividing by max_value

            # Normalizing: 
            # We do this because sparce matrices (filled with 0s) are much easier on memory 
            # Since we subtract by half the max_value, we'll never be negative 
            # Values are forced into range between 0, and highest is max_value/2
            # We also divide by max_value to set them between 0 and 1 
            
            # Uncomment the following to see this in action:
            #not_normalized_image_data= ndimage.imread(image).astype(float)
            #print(not_normalized_image_data)
            #print(image_data)
            
            #if image_data.shape != (image_size, image_size):
            #    raise Exception('Unexpected image shape: %s' % str(image_data.shape))
            #dataset[num_images, :, :] = image_data
            #num_images = num_images + 1
            
            if image_data.shape != (image_size, image_size):
            # Is it the right shape (A 28 x 28 array) 
                raise Exception('Unexpected image shape: %s' % str(image_data.shape))


            print('Appending {} to dataset.'.format(image))
            # Tell us whats about to happen

            dataset[num_images, :, :] = image_data
            # Then do it

            num_images = num_images + 1
            # Update the counter
        
        
        except IOError as e:
            print('Could not read:', image_file, ':', e, ', skipping.')
        
        
    dataset = dataset[0:num_images, :, :]
    # Final dataset better be only as long as the images we appened

    if num_images < min_num_images:
    # Lets make sure we got the minimum number of images we wanted
        raise Exception('Many fewer images than expected: %d < %d' %
                        (num_images, min_num_images))
        # Raise an exception if not
        
            
    print('\n {} Dataset complete \n Mean: {} \n Standard Deviation: {}'.format(dataset.shape, 
                                                                              np.mean(dataset), np.std(dataset)))
    # Some descriptive stats on the new tensor 
    
    return dataset

data_folder()
dataset = load_letter(test_folders[2], 5)

Creating a 1873 x 28 x 28 tensor to fill with normalized values between 0 and 255.0
On image 0.
Appending MDEtMDEtMDAudHRm.png to dataset.
On image 1.
Appending MDRiXzA4LnR0Zg==.png to dataset.
On image 2.
Appending MjAwcHJvb2Ztb29uc2hpbmUgcmVtaXgudHRm.png to dataset.
On image 3.
Appending MlJlYmVsc0RldXgtQmxhY2sub3Rm.png to dataset.
On image 4.
Appending MlRvb24gU2hhZG93LnR0Zg==.png to dataset.
On image 5.
Appending MlRvb24yIFNoYWRvdy50dGY=.png to dataset.
On image 6.
Appending MTAuMTUgU2F0dXJkYXkgTmlnaHQgQlJLLnR0Zg==.png to dataset.
On image 7.
Appending MTFTMDEgQmxhY2sgVHVlc2RheSBPZmZzZXQudHRm.png to dataset.
On image 8.
Appending MTggSG9sZXMgQlJLLnR0Zg==.png to dataset.
On image 9.
Appending MTh0aENlbnR1cnkudHRm.png to dataset.
On image 10.
Appending MTIgV2FsYmF1bSBJdGFsaWMgMTMyNjMudHRm.png to dataset.
On image 11.
Appending MTJTYXJ1WWVsbG93Rm9nLnR0Zg==.png to dataset.
On image 12.
Appending Nng3b2N0IEFsdGVybmF0ZSBFeHRyYUxpZ2h0LnR0Zg==.png to dataset.
On image 13.
Appending Nng3b2N

Pickle stuff!

In [ ]:
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letter(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)